In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\Predictive Maintenance\\Predictive_Maintenance_With_MLops'

In [5]:
from dotenv import load_dotenv
load_dotenv()
MLFLOW_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

In [12]:
# define entity 

from dataclasses import dataclass 
from pathlib import Path


@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    test_y: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config = self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN


            FileOperations.create_directories([config.root_dir])


            model_evaluation_config = ModelEvaluationConfig(
                 root_dir = config.root_dir,
                 test_data_path= config.test_data_path,
                 test_y = config.test_y,
                 model_path = config.model_path,
                 all_params = params,
                 metric_file_name = config.metric_file_name,
                 target_column= schema.name,
                 mlflow_uri= MLFLOW_URI,
            )

            return model_evaluation_config

    

In [9]:
# component 


import os 
import json
from pathlib import Path
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow 
import mlflow.sklearn 
import numpy as np 
import joblib 

In [25]:


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config 


    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2 
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        test_data.dropna(subset=[self.config.target_column], inplace=True)
        test_y = pd.read_csv(self.config.test_y)
        #test_y = pd.read_csv(self.config.test_y).values
        #print(test_y)


        model = joblib.load(self.config.model_path)


        test_x = test_data.drop([self.config.target_column], axis = 1)
        #test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            print(predicted_qualities)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)


            # Saving metrics as local 
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            FileOperations.save_json(path = Path(self.config.metric_file_name), data = scores)


            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store 
            if tracking_url_type_store == "file":


                # Register the model 
                # There are other ways to use the model registry, which depends on the use case,
                # please refer to the doc for more information:
                # http://mlflow.org/docs/latest/model-registry-.html#api-workflow
                mlflow.sklearn.log_model(model, 'model', registered_model_name= "ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
                

            

In [26]:
# Initializing my pipeline 

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    
    print("An error occurred:", str(e))

[2023-10-31 14:38:15,390: INFO: yaml file: config\config.yaml loaded successfully]
[2023-10-31 14:38:15,393: INFO: yaml file: params.yaml loaded successfully]
[2023-10-31 14:38:15,398: INFO: yaml file: schema.yaml loaded successfully]
[2023-10-31 14:38:15,409: INFO: created directory at: artifacts]
[2023-10-31 14:38:15,411: INFO: created directory at: artifacts/model_evaluation]
[104.65851265 106.79422371 105.840662   ... 106.07658514 103.81633247
 107.04907942]
[2023-10-31 14:38:16,001: INFO: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/10/31 14:38:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 5
Created version '5' of model 'ElasticnetModel'.
